# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/acordaos-selecionada-filtrados-500.csv', sep = '|')
df['filtrado_500']=df['filtrado_500'].astype(str)
df.head()

,acordao,areas,filtrado_500
0,297/2016-P,Responsabilidade,conta conta gerência instituto nacional seguro...
1,366/2016-P,Finanças Públicas,senado petróleo petróleo regime aduaneiro expo...
2,944/2016-P,Responsabilidade,tribunal conta plenário embargo declaração aco...
3,30/2016-P,Direito Processual,embargo declaração inss recorrente marco antôn...
4,55/2016-P,Pessoal,senac senac senac empresa senac giselle araújo...


In [2]:
df.shape

(10524, 3)

In [3]:
from sklearn.preprocessing import LabelBinarizer

areas = df.groupby(['areas']).groups.keys()
lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Competência do TCU,Contrato Administrativo',
       'Competência do TCU,Contrato Administrativo,Responsabilidade',
       'Competência do TCU,Convênio',
       'Competência do TCU,Convênio,Direito Processual,Licitação',
       'Competência do TCU,Convênio,Gestão Administrativa',
       'Competência do TCU,Convênio,Responsabilidade',
       'Competência do TCU,Desestatização',
       'Competência do TCU,Direito Processual',
       'Competência do TCU,Direito Processual,Licitação',
       'Competência do TCU,Direito Processual,Pessoal',
       'Competência do TCU,Direito Processual,Responsabilidade',
       'Competência do TCU,Finanças Públicas',
       'Competência do TCU,Finanças Públicas,Licitação',
       'Competência do TCU,Finanças Públicas,Responsabilidade',
       'Competência do TCU,Gestão Administrativa',
       'Competência do TCU,Licitação',
       'Competência do TCU,Licitação,Pessoal',
       'Competência do TCU,Licitação,Responsabilidade',
   

In [4]:
y = lbArea.transform(df['areas'])
y.shape

(10524, 91)

In [5]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 35000
limite_texto = 500
dim_vetor = 100

tokenizer = Tokenizer(num_words=vocabulario)
tokenizer.fit_on_texts(df['filtrado_500'])

sequences = tokenizer.texts_to_sequences(df['filtrado_500'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 33339 unique tokens.


In [6]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (10524, 500)


In [7]:
x.shape, y.shape

((10524, 500), (10524, 91))

In [8]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('../externos/model.txt')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, dim_vetor))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model:
        embedding_matrix[i] = model[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

Vocabulario: 33339
Encontrados no modelo: 24576 = 73.71546837037704


# Treinamento

In [10]:
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, Dense, Embedding
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(SimpleRNN(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0318 14:09:53.550865 140394614253376 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0318 14:09:53.673664 140394614253376 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0318 14:09:53.675789 140394614253376 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0318 14:09:53.682559 140394614253376 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprec

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          3500000   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 256)               91392     
_________________________________________________________________
dense_1 (Dense)              (None, 91)                23387     
Total params: 3,614,779
Trainable params: 3,614,779
Non-trainable params: 0
_________________________________________________________________
Train on 8419 samples, validate on 2105 samples
Epoch 1/20
8419/8419 [==============================] - 55s 6ms/step - loss: 2.2745 - categorical_accuracy: 0.3647 - val_loss: 2.1410 - val_categorical_accuracy: 0.3059
Epoch 2/20
8419/8419 [==============================] - 55s 7ms/step - loss: 2.0501 - categorical_accuracy: 0.4316 - val_loss: 1.9915 - val_categorical_accuracy: 0.3601
Epoch 3/20

In [11]:
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, Dense, Embedding
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 100)          3500000   
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_2 (Dense)              (None, 91)                23387     
Total params: 3,797,563
Trainable params: 297,563
Non-trainable params: 3,500,000
_________________________________________________________________
Train on 8419 samples, validate on 2105 samples
Epoch 1/20
8419/8419 [==============================] - 120s 14ms/step - loss: 1.8480 - categorical_accuracy: 0.5033 - val_loss: 1.5487 - val_categorical_accuracy: 0.5506
Epoch 2/20
8419/8419 [==============================] - 119s 14ms/step - loss: 1.5016 - categorical_accuracy: 0.5859 - val_loss: 1.5559 - val_categorical_accuracy: 0.5506


In [12]:
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, Dense, Embedding
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 100)          3500000   
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_3 (Dense)              (None, 91)                23387     
Total params: 3,797,563
Trainable params: 3,797,563
Non-trainable params: 0
_________________________________________________________________
Train on 8419 samples, validate on 2105 samples
Epoch 1/20
8419/8419 [==============================] - 133s 16ms/step - loss: 1.8350 - categorical_accuracy: 0.5029 - val_loss: 1.6978 - val_categorical_accuracy: 0.5249
Epoch 2/20
8419/8419 [==============================] - 132s 16ms/step - loss: 1.4610 - categorical_accuracy: 0.5997 - val_loss: 1.6987 - val_categorical_accuracy: 0.4945
Epoch 